In [ ]:
import json
import requests
from collections import defaultdict

In [ ]:
### current balance
def get_btc_balance(address):
    """
    Retrieves the Bitcoin balance for a given BTC address.

    Parameters:
    - address (str): The BTC address for which the balance is to be checked.

    Returns:
    - float: The Bitcoin balance of the provided address.

    Example:
    >>> get_btc_balance("1A1zP1eP5QGefi2DMPTfTL5SLmv7DivfNa")
    50.0

    Note:
    The balance is fetched from the blockchain.info API and is returned in satoshis,
    so the function converts it to BTC before returning.
    """
    url = f"https://blockchain.info/q/addressbalance/{address}"
    response = requests.get(url)
    btc_balance = int(response.text) / 1e8
    return btc_balance

def get_eth_balance(address, ETHERSCAN_API_KEY):
    """
    Retrieves the Ether balance for a given Ethereum address.

    Parameters:
    - address (str): The Ethereum address for which the balance is to be checked.
    - ETHERSCAN_API_KEY (str): The API key for the Etherscan API.

    Returns:
    - float: The Ether balance of the provided address.

    Example:
    >>> get_eth_balance("0x742d35Cc6634C0532925a3b844Bc454e4438f44e", "YOUR_API_KEY")
    1500.0

    Note:
    The balance is fetched from the etherscan.io API and is returned in wei,
    so the function converts it to Ether before returning.
    """
    url = f"https://api.etherscan.io/api?module=account&action=balance&address={address}&tag=latest&apikey={ETHERSCAN_API_KEY}"
    response = requests.get(url)
    data = response.json()
    eth_balance = int(data['result']) / 1e18
    return eth_balance

def get_bsc_balance(address, BSCSCAN_API_KEY):
    """
    Retrieves the BNB balance for a given Binance Smart Chain address.

    Parameters:
    - address (str): The Binance Smart Chain address for which the balance is to be checked.
    - BSCSCAN_API_KEY (str): The API key for the BscScan API.

    Returns:
    - float: The BNB balance of the provided address.

    Example:
    >>> get_bsc_balance("0xbb4CdB9CBd36B01bD1cBaEBF2De08d9173bc095c", "YOUR_API_KEY")
    200.0

    Note:
    The balance is fetched from the bscscan.com API and is returned in wei,
    so the function converts it to BNB before returning.
    """
    url = f"https://api.bscscan.com/api?module=account&action=balance&address={address}&tag=latest&apikey={BSCSCAN_API_KEY}"
    response = requests.get(url)
    data = response.json()
    bsc_balance = int(data['result']) / 1e18
    return bsc_balance

def get_token_balance_eth(address, contract_address, ETHERSCAN_API_KEY):
    """
    Retrieves the token balance for a given Ethereum address and token contract.

    Parameters:
    - address (str): The Ethereum address for which the token balance is to be checked.
    - contract_address (str): The Ethereum contract address of the token.
    - ETHERSCAN_API_KEY (str): The API key for the Etherscan API.

    Returns:
    - float or None: The token balance of the provided address. Returns None if data fetch was unsuccessful.

    Example:
    >>> get_token_balance_eth("0x...", "0x...", "YOUR_API_KEY")
    150.0

    Note:
    The function assumes the token has 18 decimals. Adjust accordingly if different.
    """
    url = f"https://api.etherscan.io/api?module=account&action=tokenbalance&contractaddress={contract_address}&address={address}&tag=latest&apikey={ETHERSCAN_API_KEY}"
    response = requests.get(url)
    data = response.json()
    if data['message'] == 'OK':
        return int(data['result']) / (10**18)
    return None

def get_token_balances_eth(address, ETHERSCAN_API_KEY):
    """
    Retrieves all token balances associated with a given Ethereum address.

    Parameters:
    - address (str): The Ethereum address for which token balances are to be checked.
    - ETHERSCAN_API_KEY (str): The API key for the Etherscan API.

    Returns:
    - dict: A dictionary containing token symbols as keys, each associated with its balance and contract address.

    Example:
    >>> get_token_balances_eth("0x...", "YOUR_API_KEY")
    {"USDT": {"balance": 500.0, "contract": "0x..."}, "MKR": {"balance": 2.0, "contract": "0x..."}}
    """
    url = f"https://api.etherscan.io/api?module=account&action=tokentx&address={address}&startblock=0&endblock=99999999&sort=asc&apikey={ETHERSCAN_API_KEY}"
    response = requests.get(url)
    data = response.json()
    balances = {}

    for tx in data['result']:
        token_symbol = tx['tokenSymbol']
        contract_address = tx['contractAddress']
        if token_symbol not in balances:
            balance_c = get_token_balance_eth(address, contract_address, ETHERSCAN_API_KEY)
            balances[token_symbol] = {'balance': balance_c, 'contract': contract_address}
    return balances

def get_token_balance_bsc(address, contract_address, BSCSCAN_API_KEY):
    """
    Retrieves the token balance for a given Binance Smart Chain address and token contract.

    Parameters:
    - address (str): The Binance Smart Chain address for which the token balance is to be checked.
    - contract_address (str): The Binance Smart Chain contract address of the token.
    - BSCSCAN_API_KEY (str): The API key for the BscScan API.

    Returns:
    - float or None: The token balance of the provided address. Returns None if data fetch was unsuccessful.

    Example:
    >>> get_token_balance_bsc("0x...", "0x...", "YOUR_API_KEY")
    1000.0

    Note:
    The function assumes the token has 18 decimals. Adjust accordingly if different.
    """
    url = f"https://api.bscscan.com/api?module=account&action=tokenbalance&contractaddress={contract_address}&address={address}&tag=latest&apikey={BSCSCAN_API_KEY}"
    response = requests.get(url)
    data = response.json()
    if data['message'] == 'OK':
        return int(data['result']) / (10**18)
    return None

def get_token_balances_bsc(address, BSCSCAN_API_KEY):
    """
    Retrieves all token balances associated with a given Binance Smart Chain address.

    Parameters:
    - address (str): The Binance Smart Chain address for which token balances are to be checked.
    - BSCSCAN_API_KEY (str): The API key for the BscScan API.

    Returns:
    - dict: A dictionary containing token symbols as keys, each associated with its balance and contract address.

    Example:
    >>> get_token_balances_bsc("0x...", "YOUR_API_KEY")
    {"CAKE": {"balance": 50.0, "contract": "0x..."}, "BUSD": {"balance": 1000.0, "contract": "0x..."}}
    """
    url = f"https://api.bscscan.com/api?module=account&action=tokentx&address={address}&startblock=1&endblock=99999999&sort=asc&apikey={BSCSCAN_API_KEY}"
    response = requests.get(url)
    data = response.json()
    balances = {}

    for tx in data['result']:
        token_symbol = tx['tokenSymbol']
        contract_address = tx['contractAddress']
        if token_symbol not in balances:
            balance_c = get_token_balance_bsc(address, contract_address, BSCSCAN_API_KEY)
            balances[token_symbol] = {'balance': balance_c, 'contract': contract_address}
    return balances

def get_token_price_coingecko(token_symbol):
    """
    Fetch the current token price from the CoinGecko API.

    Parameters:
    - token_symbol (str): The symbol of the token.

    Returns:
    - float or None: The price of the token in USD or None if not found.

    Example:
    >>> get_token_price_coingecko('btc')
    50000.23
    """
    url = f"https://api.coingecko.com/api/v3/simple/price?ids={token_symbol}&vs_currencies=usd"
    response = requests.get(url)
    data = response.json()

    if token_symbol in data and 'usd' in data[token_symbol]:
        return data[token_symbol]['usd']
    else:
        return None

def get_coinpaprika_coins(json_paprika):
    """
    Fetch a dictionary of token symbols to their IDs from CoinPaprika.

    Returns:
    - dict: A dictionary with token symbols as keys and their corresponding CoinPaprika IDs as values.

    Example:
    >>> get_coinpaprika_coins()
    {"btc": "bitcoin", "eth": "ethereum"}
    """
    return {i['symbol']: i['id'] for i in json_paprika}

def get_token_price_coinpaprika(token_symbol, coinpaprika_coins):
    """
    Fetch the current token price from CoinPaprika.

    Parameters:
    - token_symbol (str): The symbol of the token.
    - coinpaprika_coins (dict): A dictionary mapping token symbols to CoinPaprika IDs.

    Returns:
    - float or None: The price of the token in USD or None if not found.
    """
    if token_symbol not in coinpaprika_coins:
        return None
    coin_id = coinpaprika_coins[token_symbol]
    url = f"https://api.coinpaprika.com/v1/tickers/{coin_id}"
    response = requests.get(url)
    data = response.json()
    return data['quotes']['USD']['price'] if 'quotes' in data and 'USD' in data['quotes'] else None

def get_coinmarketcap_coins():
    """
    Fetch a dictionary of token symbols to their slug from CoinMarketCap.

    Returns:
    - dict: A dictionary with token symbols as keys and their corresponding CoinMarketCap slug as values.

    Note:
    The function uses a hardcoded API key which should be securely managed.
    """
    headers = {
        'X-CMC_PRO_API_KEY': '72abe7bf-eb40-4098-a3f0-95697f635c99',
    }
    url = f"https://pro-api.coinmarketcap.com/v1/cryptocurrency/map"
    response = requests.get(url, headers=headers).json()
    return {i['symbol']:i['slug'] for i in response['data']}

def get_token_price_coinmarketcap(token_symbol):
    """
    Fetch the current token price from CoinMarketCap.

    Parameters:
    - token_symbol (str): The symbol of the token.

    Returns:
    - float or None: The price of the token in USD or None if not found.

    Note:
    The function uses a hardcoded API key which should be securely managed.
    """
    headers = {
        'X-CMC_PRO_API_KEY': '72abe7bf-eb40-4098-a3f0-95697f635c99',
    }
    url = f"https://pro-api.coinmarketcap.com/v1/cryptocurrency/quotes/latest?symbol={token_symbol.upper()}"
    response = requests.get(url, headers=headers).json()

    if 'data' in response and token_symbol.upper() in response['data'] and 'quote' in response['data'][token_symbol.upper()] and 'USD' in response['data'][token_symbol.upper()]['quote']:
        return response['data'][token_symbol.upper()]['quote']['USD']['price']
    else:
        return None

def get_token_price(token_symbol, json_paprika):
    """
    Fetch the current token price from multiple sources in order: CoinGecko, CoinPaprika, and CoinMarketCap.

    Parameters:
    - token_symbol (str): The symbol of the token.

    Returns:
    - float or None: The price of the token in USD or None if not found.
    """
    coinpaprika_coins = get_coinpaprika_coins(json_paprika)
    # CoinGecko and CoinPaprika use lowercase token symbols
    token_symbol_lower = token_symbol.lower()
    # Try to get the price from each data source until a price is found
    price = get_token_price_coingecko(token_symbol_lower)
    if price is None:
        price = get_token_price_coinpaprika(token_symbol_lower, coinpaprika_coins)
    if price is None:
        price = get_token_price_coinmarketcap(token_symbol)
    return price

In [ ]:
def convert_balances_to_usd(address, ETHERSCAN_API_KEY, BSCSCAN_API_KEY, balances, json_paprika):
    """
    Convert token balances to their corresponding value in USD.

    Parameters:
    - address (str): The address for which the balances are being converted.
    - balances (dict): Dictionary containing tokens and their balances.

    Returns:
    - dict: A dictionary with token symbols as keys and their USD values as values.

    Example:
    >>> balances = {'ETH': {'balance': 2, 'contract': '0x123...'}}
    >>> convert_balances_to_usd('0xabc...', balances)
    {'ETH': 5000.0}
    """
    balances_usd = {}

    for token, info in balances.items():
        balance = info['balance']

        if token.lower() == 'wbnb':
            token = 'BNB'
            balance = get_bsc_balance(address, BSCSCAN_API_KEY)
        elif token.lower() == 'weth':
            token = 'ETH'
            balance = get_eth_balance(address, ETHERSCAN_API_KEY)

        price = get_token_price(token.lower(), json_paprika)  # CoinGecko and CoinPaprika use lowercase token symbols
        if price is not None:
            balances_usd[token] = balance * price
        else:
            # Indicates we couldn't fetch the price for this token
            balances_usd[token] = None

    return balances_usd

### transactions

def get_btc_transactions(address):
    """
    Fetch BTC transactions for a given address from the Blockchain.info API.

    Parameters:
    - address (str): The BTC address.

    Returns:
    - list: A list of transaction objects.
    """
    url = f"https://blockchain.info/rawaddr/{address}"
    response = requests.get(url)
    data = response.json()
    return data['txs']

def get_eth_transactions(address):
    """
    Fetch ETH transactions for a given address from the Etherscan API.

    Parameters:
    - address (str): The ETH address.

    Returns:
    - list: A list of transaction objects.

    Note:
    The function uses a hardcoded API key which should be securely managed.
    """
    url = f"https://api.etherscan.io/api?module=account&action=txlist&address={address}&startblock=0&endblock=99999999&sort=asc&apikey={ETHERSCAN_API_KEY}"
    response = requests.get(url)
    data = response.json()
    return data['result']

def get_bsc_transactions(address):
    """
    Fetch BSC transactions for a given address from the BscScan API.

    Parameters:
    - address (str): The BSC address.

    Returns:
    - list: A list of transaction objects.

    Note:
    The function uses a hardcoded API key which should be securely managed.
    """
    url = f"https://api.bscscan.com/api?module=account&action=txlist&address={address}&startblock=1&endblock=99999999&page=1&offset=10&sort=asc&apikey={BSCSCAN_API_KEY}"
    response = requests.get(url)
    data = response.json()
    return data['result']

def get_token_info_etherscan(ETHERSCAN_API_KEY, contract_address):
    """
    Fetch information about a token using its contract address from Etherscan.

    Parameters:
    - ETHERSCAN_API_KEY (str): The API key for accessing Etherscan.
    - contract_address (str): The contract address of the token.

    Returns:
    - dict: A dictionary containing information about the token.

    Note:
    The function uses a provided API key which should be securely managed.
    """
    url = f"https://api.etherscan.io/api?module=token&action=tokeninfo&contractaddress={contract_address}&apikey={ETHERSCAN_API_KEY}"
    response = requests.get(url)
    data = response.json()
    return data

In [ ]:
paprika_coins = requests.get('https://api.coinpaprika.com/v1/tickers').json()
# paprika_coins

In [ ]:
# Your API keys
ETHERSCAN_API_KEY = "B1IKUF8DH2CATPWWBGKQDECVYI234YB5MW"
BSCSCAN_API_KEY = "IK74KUPCN15ZKHCYIJPHJJGD9TF7K2UE81"

In [ ]:
# Obtiene la lista de criptomonedas de Coinpaprika y CoinMarketCap
coinpaprika_coins = get_coinpaprika_coins(paprika_coins)
coinmarketcap_symbols = get_coinmarketcap_coins()

In [ ]:
eth_address = "0x770c04f124dfca403dcea65427cb60d262cde7a2"
bsc_address = "0x770c04f124dfca403dcea65427cb60d262cde7a2"
get_eth_balance(eth_address, ETHERSCAN_API_KEY)

0.19316991456210805

In [ ]:
get_bsc_balance(eth_address, BSCSCAN_API_KEY)

0.009315331301992672

In [ ]:
# Obtiene el balance de tokens de la dirección de Ethereum
eth_token_balances = get_token_balances_eth(eth_address, ETHERSCAN_API_KEY)
eth_token_balances

{'SOS': {'balance': 0.0,
  'contract': '0x3b484b82567a09e2588a13d54d032153f0c0aee0'},
 'LOOKS': {'balance': 0.0,
  'contract': '0xf4d2888d29d722226fafa5d9b24f9164c092421e'},
 'TKNFY': {'balance': 2263.1767101,
  'contract': '0xa6dd98031551c23bb4a2fbe2c4d524e8f737c6f7'},
 'X2Y2': {'balance': 0.0,
  'contract': '0x1e4ede388cbc9f4b5c79681b7f94d36a11abebc9'},
 'KongzDAO.io': {'balance': 5555.0,
  'contract': '0x6fbb29ec3946f6e1721d9c963d4a26ebd7802d27'},
 'WETH': {'balance': 0.0,
  'contract': '0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2'},
 'BEND': {'balance': 6484.4400000000005,
  'contract': '0x0d02755a5700414b26ff040e1de35d337df56218'},
 'CrypToadzDAO.io': {'balance': 5555.0,
  'contract': '0x22ac7b1699abb9466662201145f701aab57577c2'},
 'APE': {'balance': 0.0,
  'contract': '0x4d224452801aced8b2f0aebe155379bb5d594381'},
 'Otherside Gas Refund (koda.family)': {'balance': 10.0,
  'contract': '0x7a4a4ebc1a4789ab721e73eff4389314c58f9f25'},
 'Otherside Gas Refund (OTHR.claims)': {'balance': 

In [ ]:
# Convierte los saldos de tokens a USD
eth_token_balances_usd = convert_balances_to_usd(eth_address, ETHERSCAN_API_KEY, BSCSCAN_API_KEY, eth_token_balances, paprika_coins)
eth_token_balances_usd

{'SOS': 0.0,
 'LOOKS': 0.0,
 'TKNFY': 0.23714450619038263,
 'X2Y2': 0.0,
 'KongzDAO.io': None,
 'ETH': None,
 'BEND': 25.78486634641835,
 'CrypToadzDAO.io': None,
 'APE': 0.0,
 'Otherside Gas Refund (koda.family)': None,
 'Otherside Gas Refund (OTHR.claims)': None,
 'Luna Community Refund (terraluna.cloud)': None,
 'mbird.fun': None,
 'VOTEFORK.COM': None,
 'LADYS': 0.0,
 'AirPEPE': 5.198585534501861e-10,
 'ZUM': 1.5865143131701543e-08,
 'AIPEPE': 0.011079927149348285,
 'PANDA': 1917591.1872699491,
 'RADDIT': 0.013192930083203831,
 'BIBI': 54.94256025504814,
 'CZ': 0.0,
 'DUCKS': 5.806084305016905,
 'FREN': 2.764927334515704,
 'SCooBi': 135.28501414818925,
 'KISS': None,
 'FBJ': None,
 'pBTC': None,
 '4GMS': None,
 'SPYOP': None,
 'CARDANO': None,
 'GENTS': None,
 'mint.fun Token': None,
 'HATE': 0.0,
 'OPPSY': None,
 'Toking High Club': None,
 'GotGVol3': None,
 'SOBACK': None,
 'PSYOOPS': None,
 'IUIU': None,
 'XDOGE': 0.0001505480747464315,
 'Visit https://uniETH.fi to claim reward'

In [ ]:
# Obtiene el balance de tokens de la dirección de Ethereum
eth_token_balances = get_token_balances_eth(eth_address, ETHERSCAN_API_KEY)
eth_token_balances

{'SOS': {'balance': 0.0,
  'contract': '0x3b484b82567a09e2588a13d54d032153f0c0aee0'},
 'LOOKS': {'balance': 0.0,
  'contract': '0xf4d2888d29d722226fafa5d9b24f9164c092421e'},
 'TKNFY': {'balance': 2263.1767101,
  'contract': '0xa6dd98031551c23bb4a2fbe2c4d524e8f737c6f7'},
 'X2Y2': {'balance': 0.0,
  'contract': '0x1e4ede388cbc9f4b5c79681b7f94d36a11abebc9'},
 'KongzDAO.io': {'balance': 5555.0,
  'contract': '0x6fbb29ec3946f6e1721d9c963d4a26ebd7802d27'},
 'WETH': {'balance': 0.0,
  'contract': '0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2'},
 'BEND': {'balance': 6484.4400000000005,
  'contract': '0x0d02755a5700414b26ff040e1de35d337df56218'},
 'CrypToadzDAO.io': {'balance': 5555.0,
  'contract': '0x22ac7b1699abb9466662201145f701aab57577c2'},
 'APE': {'balance': 0.0,
  'contract': '0x4d224452801aced8b2f0aebe155379bb5d594381'},
 'Otherside Gas Refund (koda.family)': {'balance': 10.0,
  'contract': '0x7a4a4ebc1a4789ab721e73eff4389314c58f9f25'},
 'Otherside Gas Refund (OTHR.claims)': {'balance': 

In [ ]:
# Convierte los saldos de tokens a USD
eth_token_balances_usd = convert_balances_to_usd(eth_address, ETHERSCAN_API_KEY, BSCSCAN_API_KEY, eth_token_balances, paprika_coins)
eth_token_balances_usd

{'SOS': 0.0,
 'LOOKS': 0.0,
 'TKNFY': 0.23714450619038263,
 'X2Y2': 0.0,
 'KongzDAO.io': None,
 'ETH': None,
 'BEND': 25.77497446125577,
 'CrypToadzDAO.io': None,
 'APE': 0.0,
 'Otherside Gas Refund (koda.family)': None,
 'Otherside Gas Refund (OTHR.claims)': None,
 'Luna Community Refund (terraluna.cloud)': None,
 'mbird.fun': None,
 'VOTEFORK.COM': None,
 'LADYS': 0.0,
 'AirPEPE': 5.198585534501861e-10,
 'ZUM': 1.5864148765495775e-08,
 'AIPEPE': 0.011078509411785144,
 'PANDA': 1923398.6078488247,
 'RADDIT': 0.013192930083203831,
 'BIBI': 54.95412401170314,
 'CZ': 0.0,
 'DUCKS': 5.803854169040673,
 'FREN': 2.76519747556187,
 'SCooBi': 135.28501414818925,
 'KISS': None,
 'FBJ': None,
 'pBTC': None,
 '4GMS': None,
 'SPYOP': None,
 'CARDANO': None,
 'GENTS': None,
 'mint.fun Token': None,
 'HATE': None,
 'OPPSY': None,
 'Toking High Club': None,
 'GotGVol3': None,
 'SOBACK': None,
 'PSYOOPS': None,
 'IUIU': None,
 'XDOGE': 0.0001505480747464315,
 'Visit https://uniETH.fi to claim reward'

In [ ]:
# Obtiene el balance de tokens de la dirección de Binance Chain
bsc_token_balances = get_token_balances_bsc(bsc_address, BSCSCAN_API_KEY)
bsc_token_balances

{'RACA': {'balance': 0.0,
  'contract': '0x12bb890508c125661e03b09ec06e404bc9289040'},
 'Def8.io': {'balance': 82445.0,
  'contract': '0x556798dd55db12562a6950ea8339a273539b0495'},
 'PowNFT.net': {'balance': 96816.55,
  'contract': '0x893c25c46bfaa9b66cd557837d32af3fe264a07b'},
 '1Gas.org': {'balance': 92280.0,
  'contract': '0xd35f9ab96d04adb02fd549ef6a576ce4e2c1d935'},
 'SHIBA_DIVIDEND_TRACKER': {'balance': 66707.0,
  'contract': '0xdddf82fb98530243fcf8d4b8dc452f918c3ac4ac'},
 'PDot.io': {'balance': 95641.0,
  'contract': '0x8bd0e87273364ebbe3482efc166f7e0d34d82c25'},
 'WBNB': {'balance': 0.0,
  'contract': '0xbb4cdb9cbd36b01bd1cbaebf2de08d9173bc095c'},
 'ABOYS': {'balance': 18000000.0,
  'contract': '0xeade8e37700a34b196142d9962858d2a137e6eb8'},
 'MKS': {'balance': 6500000.0,
  'contract': '0x64f2c2aa04755507a2ecd22ceb8c475b7a750a3a'},
 'swap8.io': {'balance': 980000.0,
  'contract': '0xcaee79616cffeb53fdda5792742a5c084f879dec'},
 'AkkSwap.io': {'balance': 3.2e-05,
  'contract': '0x

In [ ]:
# Convierte los saldos de tokens a USD
bsc_token_balances_usd = convert_balances_to_usd(bsc_address, ETHERSCAN_API_KEY, BSCSCAN_API_KEY, bsc_token_balances, paprika_coins)
bsc_token_balances_usd

{'RACA': 0.0,
 'Def8.io': None,
 'PowNFT.net': None,
 '1Gas.org': None,
 'SHIBA_DIVIDEND_TRACKER': None,
 'PDot.io': None,
 'BNB': 2.2407124905383955,
 'ABOYS': None,
 'MKS': None,
 'swap8.io': None,
 'AkkSwap.io': None,
 'DBT': 0.0,
 'Air Pepe': None,
 'DL': None,
 'CAPO': 0.003092590333308182,
 'HENRY': None}